In [ ]:
import polars as pl
import pyodbc

In [ ]:
def read_access(file, table):
    conn_str = r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=" + file
    conn = pyodbc.connect(conn_str)

    df = pl.read_database(query=f"SELECT * FROM {table}", connection=conn)
    conn.close()
    return df

### Querie Pagos

In [ ]:
def import_f1():
    df_desembolsos = read_access(fl_desembolsos_p1, "DATOS")
    df_desembolsos_bmxt = read_access(fl_desembolsos_p1_bmxt, "DATOS")
    df_desembolsos_fianzas = read_access(fl_desembolsos_p1_fianzas, "DATOS")

    aux = pl.concat([
        df_desembolsos, 
        df_desembolsos_bmxt, 
        df_desembolsos_fianzas
        ], rechunk=True)
    
    df_pagos_f1 = (
        aux.select(
            pl.col("DESC_INDICADOR").alias("Producto"),
            pl.col("ESTATUS_RECUPERACION"),
            pl.col("FECHA_APERTURA").alias("Fecha de Apertura"),
            pl.col("FECHA_GARANTIA_HONRADA"),
            pl.col("FECHA_PRIMER_INCUMPLIMIENTO"),
            pl.col("FECHA_REGISTRO_ALTA").alias("Fecha Registro Alta"),
            pl.col("INTERMEDIARIO_ID"),
            pl.col("MONEDA_ID"),
            pl.col("NOMBRE_EMPRESA").alias("Empresa / Acreditado (Descripción)"),
            pl.col("NUMERO_CREDITO"),
            pl.col("PORCENTAJE_GARANTIZADO"),
            pl.col("PROGRAMA_ID"),
            pl.col("PROGRAMA_ORIGINAL"),
            pl.col("RAZON_SOCIAL").alias("Razón Social (Intermediario)"),
            pl.col("RFC_EMPRESA").alias("RFC Empresa / Acreditado"),
            pl.col("TIPO_CREDITO_ID"),
            pl.col("TIPO_GARANTIA_ID"),
            pl.col("TIPO_PERSONA"),
            pl.col("MONTO_CREDITO_MN (SUMA)").alias("Monto _Credito_Mn")
        )
    )
    return df_pagos_f1

def import_f2():
    df_desembolsos = read_access(fl_desembolsos_p2, "DATOS")
    df_desembolsos_bmxt = read_access(fl_desembolsos_p2_bmxt, "DATOS")
    df_desembolsos_fianzas = read_access(fl_desembolsos_p2_fianzas, "DATOS")

    aux = pl.concat([
        df_desembolsos, 
        df_desembolsos_bmxt, 
        df_desembolsos_fianzas
        ], rechunk=True)
    
    df_pagos_f2 = (
        aux.select(
            pl.col("DESC_INDICADOR").alias("Producto"),
            pl.col("FECHA_CONSULTA"),
            pl.col("FECHA_REGISTRO").alias("MIN Fecha_Registro"),
            pl.col("HISTORICO").alias("MAX Historico"),
            pl.col("INDICADOR_ID").alias("Producto ID"),
            pl.col("INTERMEDIARIO_ID"),
            pl.col("MONEDA_ID"),
            pl.col("NUMERO_CREDITO"),
            pl.col("PAGO_ID").alias("Pago ID"),
            pl.col("DFI_INTERESES_MORATORIOS (SUMA)").alias("SUM Intereses Moratorios"),
            pl.col("INTERES_DESEMBOLSO (SUMA)").alias("SUM Interes_Desembolso"),
            pl.col("MONTO_DESEMBOLSO (SUMA)").alias("SUM Monto_Desembolso")
        )
    )
    return df_pagos_f2



In [ ]:
def genera_concatenado(df):
    result = (df.with_columns(
                (pl.col("INTERMEDIARIO_ID") + pl.col("NUMERO_CREDITO"))
                .alias("Concatenado"))
            )
    
    return result


def genera_tpro_clave(df):
    result = df.with_columns(
        pl.when((pl.col("PROGRAMA_ID")>=32000)&(pl.col("PROGRAMA_ID")<=32100))
        .then(pl.col("PROGRAMA_ID"))
        .when((pl.col("PROGRAMA_ID")==3976)&(pl.col("PROGRAMA_ORIGINAL")==31415))
        .then(pl.col("PROGRAMA_ID"))
        .when((pl.col("PROGRAMA_ID")==33366)&(pl.col("PROGRAMA_ORIGINAL")==33842))
        .then(pl.col("PROGRAMA_ID"))
        .when((pl.col("PROGRAMA_ID").is_in([3536, 3537, 3539, 3542,3544, 3545, 3546,3547,3548,3549,3550, 3553, 3555, 3558,3559, 3560, 3564,3566]))&(pl.col("PROGRAMA_ORIGINAL")==3200))
        .then(pl.col("PROGRAMA_ID"))
        .when(pl.col("PROGRAMA_ORIGINAL")==3999)
        .then("PROGRAMA_ID")
        .otherwise(pl.col("PROGRAMA_ORIGINAL")).alias("TPRO_CLAVE")
    )
    return result


def genera_pagadas_global_inter(df1, df2):
    result = (df2.join(df1, on="Concatenado", how="left")
        .rename({
            "MIN Fecha_Registro": "MIN_Fecha_Registro",
            "SUM Monto_Desembolso":"Monto_Desembolsado",
            "SUM Interes_Desembolso": "Interes_Desembolso",
            "SUM Intereses Moratorios": "Interes_Moratorios"
        })
        .select([
            "Concatenado",
            "FECHA_CONSULTA", 
            "INTERMEDIARIO_ID", 
            "NUMERO_CREDITO", 
            "Producto", 
            "Pago ID",
            "Razón Social (Intermediario)",
            "MIN_Fecha_Registro",
            "FECHA_GARANTIA_HONRADA",
            "TPRO_CLAVE", 
            "PROGRAMA_ORIGINAL", 
            "PROGRAMA_ID", 
            "Monto _Credito_Mn", 
            "MONEDA_ID",
            "Fecha de Apertura",
            "TIPO_GARANTIA_ID",
            "TIPO_PERSONA",
            "RFC Empresa / Acreditado",
            "Monto_Desembolsado",
            "Interes_Desembolso",
            "Interes_Moratorios",
            "PORCENTAJE_GARANTIZADO",
            "TIPO_CREDITO_ID",
            "ESTATUS_RECUPERACION",
            "Empresa / Acreditado (Descripción)",
            "Fecha Registro Alta"
        ])
        .with_columns(pl.col("TIPO_GARANTIA_ID").fill_null(999))
    )
    return result

def genera_pagadas_global_vf(df):
    # Requiere que se hayan importado los catálogos
    result = (df
    .join(programa.select(['PROGRAMA_ID', 'AGRUPAMIENTO_ID', 'ESQUEMA', 'SUBESQUEMA']), on="PROGRAMA_ID", how='left')
        .join(agrupamiento, on='AGRUPAMIENTO_ID', how='left')
        .join(udis, left_on="Fecha de Apertura", right_on="Fecha_Paridad", how='left')
        .join(tipo_credito.select(['Tipo_Credito_ID', 'NR_R']), left_on='TIPO_CREDITO_ID', right_on="Tipo_Credito_ID", how='left')
        .join(tipo_garantia.select(['Tipo_garantia_ID', 'CSG']), left_on='TIPO_GARANTIA_ID', right_on='Tipo_garantia_ID', how='left')
        .join(sfc.select(['Intermediario_Id', 'CLAVE_CREDITO', 'FONDOS_CONTRAGARANTIA']), left_on=['INTERMEDIARIO_ID', 'NUMERO_CREDITO'], right_on=['Intermediario_Id', 'CLAVE_CREDITO'], how='left')
        )

    # Complementa
    result = (result
        .with_columns(pl.when(pl.col('MONEDA_ID')==54)
                    .then(tdc).otherwise(1).alias("TC"))
        .with_columns(pl.when(pl.col("Monto _Credito_Mn")<=(900000*pl.col("Paridad_Peso")))
                    .then(0).otherwise(1).alias("MM_UDIS"))
        .with_columns(pl.when(pl.col("FONDOS_CONTRAGARANTIA")=="SF")
                    .then(pl.lit("SF")).otherwise(pl.lit("CF")).alias("CSF"))
        )
    
    return result


def complementa_pagadas_global_vf(df):
    aux = df.select(['Monto_Desembolsado', 'Interes_Moratorios', 'Interes_Desembolso']).sum_horizontal(ignore_nulls=True)

    result = (df
        .with_columns((pl.col("Monto_Desembolsado") * pl.col("TC") * -1).alias("Monto_Desembolso_Mn"))
        .with_columns((pl.col("Interes_Desembolso") * pl.col("TC") * -1).alias("Interes_Desembolso_Mn"))
        .with_columns((pl.col("Interes_Moratorios") * pl.col("TC") * -1).alias("Interes_Moratorios_Mn"))
        .with_columns((aux).alias("Monto_Pagado_Mn"))
    )

    return result

def genera_validador(df):
    result = (df
        .group_by(["MAX Historico", "Producto", "MONEDA_ID"])
        .agg(pl.col("SUM Monto_Desembolso").sum(),
            pl.col("SUM Interes_Desembolso").sum(),
            pl.col("SUM Intereses Moratorios").sum()
            )
        )
    return result

def genera_pagadas_global_bancomext(df):
    result = (df.filter(
        (pl.col("Producto") == "GARANTIAS BANCOMEXT") |
        (pl.col("Producto") == "GARANTIAS SHF/LI FINANCIERO") |
        (pl.col("Producto") == "GARANTIAS BANSEFI") |
        (pl.col("Producto").is_null())
    ))
    return result

def genera_pagadas_global_sin_bancomext(df):
    result = (df.filter(
        (pl.col("Producto") != "GARANTIAS BANCOMEXT") &
        (pl.col("Producto") != "GARANTIAS SHF/LI FINANCIERO") &
        (pl.col("Producto") != "GARANTIAS BANSEFI")
    ))
    return result

def genera_valida_base_pagos_mn(pagadas_global_vf):
    valida_base_pagos_mn = (pagadas_global_vf
    .group_by('Producto')
    .agg(pl.col('Monto_Desembolso_Mn').sum().alias('Monto_Desembolso_Mn_Suma'),
        pl.col('Interes_Desembolso_Mn').sum().alias('Interes_Desembolso_Mn_Suma'),
        pl.col('Interes_Moratorios_Mn').sum().alias('Interes_Moratorios_Mn_Sum')
        )
    )
    return valida_base_pagos_mn

def genera_valida_base_pagos(pagadas_global_vf):
     valida_base_pagos = (pagadas_global_vf
     .group_by('Producto')
     .agg(pl.col('Monto_Desembolsado').sum().alias('Monto_Desembolsado_Suma'),
          pl.col('Interes_Desembolso').sum().alias('Interes_Desembolso_Suma'),
          pl.col('Interes_Moratorios').sum().alias('Interes_Moratorios_Sum')
          )
     )
     return valida_base_pagos

#### Querie Recuperadas

#### Querie UnionFlujos